
# Quantum Tomography with QGL, Auspex and QuantumTomography.jl

The goal of tomography in quantum information processing is to asses how well states are being created and quantum processes implemented in a given system.  In this notebook we'll outline the process of creating and modifying sequences with `QGL` to carry out state and process tomography.  `Auspex` will orchestrate the data taking process and `QuantumTomography.jl` will be used to reconstruct the state or process from the data.  At the moment this reconstruction process happens in a Julia programming environment so this notebook will focus almost entirely on creating the experiments.

We'll start in the usual way by importing and setting things up in `QGL`.

In [1]:
from QGL import *

In [2]:
cl = ChannelLibrary()
q1 = QubitFactory("q1")
q2 = QubitFactory("q2")

We should now have a qubit object in the python namespace that reflects the parameters in our YAML files

In [3]:
q1

Qubit('q1')

In [4]:
q1.pulse_params

{'cutoff': 2,
 'drag_scaling': 0,
 'length': 2e-08,
 'pi2Amp': 0.5,
 'piAmp': 1.0,
 'shape_fun': <function QGL.PulseShapes.gaussian>,
 'sigma': 5e-09}

## Sequence creation

`QGL` provides some basic routines for standard state and process tomography.  We'll take a closer look at the sequence creation process.

At a high level, there are two helper functions you're most likely to use: `state_tomo` and `process_tomo`:

### state tomography

In [5]:
?state_tomo

Let's see what we can do with as far as basic state tomography goes:

In [6]:
seqs = state_tomo([Id(q1)], (q1,), numPulses=4)

In [7]:
seqs

[[Id(q1), Id(q1), MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378>)],
 [Id(q1), X90(q1), MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378>)],
 [Id(q1), Y90(q1), MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378>)],
 [Id(q1), X(q1), MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378>)]]

We can see above, the helper function created a tomography experiment for the identity gate using projections onto the $Z$, $Y$, $X$ and $-Z$ axes.  We could asked for a different set of projections by changing the `numPulses` parameter:

In [8]:
seqs = state_tomo([Id(q1)], (q1,), numPulses=6);seqs

[[Id(q1), Id(q1), MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378>)],
 [Id(q1), X90(q1), MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378>)],
 [Id(q1), X90m(q1), MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378>)],
 [Id(q1), Y90(q1), MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378>)],
 [Id(q1), Y90m(q1), MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378>)],
 [Id(q1), X(q1), MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378>)]]

This set of experiments gives the full set of projections onto the $\pm \{X, Y, Z\}$ axes.  Of course, the identity gate is a simple thing to do but you can add any thing you'd like to the list of gates to run before the tomography blocks.

In [9]:
seqs = state_tomo([Id(q1), X(q1)], (q1,), numPulses=4);seqs

[[Id(q1),
  X(q1),
  Id(q1),
  MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378>)],
 [Id(q1),
  X(q1),
  X90(q1),
  MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378>)],
 [Id(q1),
  X(q1),
  Y90(q1),
  MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378>)],
 [Id(q1),
  X(q1),
  X(q1),
  MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378>)]]

The concepts extend to multi-qubit tomograpy as well.  Here we just pass two qubits to the function instead of one and we reconstruct the 16 different projections necessary for reconstruction.

In [10]:
seqs = state_tomo([Id(q1)], (q1,q2), numPulses=4);len(seqs)

16

In [11]:
seqs[0:5]

[[Id(q1),
  Id(q1)⊗ Id(q2),
  MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378>)⊗ MEAS(M-q2, shape_fun=<function autodyne at 0x112a47378>)],
 [Id(q1),
  Id(q1)⊗ X90(q2),
  MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378>)⊗ MEAS(M-q2, shape_fun=<function autodyne at 0x112a47378>)],
 [Id(q1),
  Id(q1)⊗ Y90(q2),
  MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378>)⊗ MEAS(M-q2, shape_fun=<function autodyne at 0x112a47378>)],
 [Id(q1),
  Id(q1)⊗ X(q2),
  MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378>)⊗ MEAS(M-q2, shape_fun=<function autodyne at 0x112a47378>)],
 [Id(q1),
  X90(q1)⊗ Id(q2),
  MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378>)⊗ MEAS(M-q2, shape_fun=<function autodyne at 0x112a47378>)]]

Here measurement is also done at the same time on both qubits.  

### process tomography

Creating experiments for process tomography is accomplished in much the same way as state tomography.

In [12]:
?process_tomo

In [13]:
# These sequences do process tomography on the Y90m gate on q1.
# In this case, we also ask for measurement records on q2 even though in theory nothing is happening.
seqs = process_tomo([Y90m(q1)], (q1,), measChans=(q1,q2))

In [14]:
seqs

[[Id(q1),
  Y90m(q1),
  Id(q1),
  MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378>)⊗ MEAS(M-q2, shape_fun=<function autodyne at 0x112a47378>)],
 [Id(q1),
  Y90m(q1),
  X90(q1),
  MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378>)⊗ MEAS(M-q2, shape_fun=<function autodyne at 0x112a47378>)],
 [Id(q1),
  Y90m(q1),
  Y90(q1),
  MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378>)⊗ MEAS(M-q2, shape_fun=<function autodyne at 0x112a47378>)],
 [Id(q1),
  Y90m(q1),
  X(q1),
  MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378>)⊗ MEAS(M-q2, shape_fun=<function autodyne at 0x112a47378>)],
 [X90(q1),
  Y90m(q1),
  Id(q1),
  MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378>)⊗ MEAS(M-q2, shape_fun=<function autodyne at 0x112a47378>)],
 [X90(q1),
  Y90m(q1),
  X90(q1),
  MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378>)⊗ MEAS(M-q2, shape_fun=<function autodyne at 0x112a47378>)],
 [X90(q1),
  Y90m(q1),
  Y90(q1),
  MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378

In [15]:
seqs = process_tomo([Y90m(q1)*X(q2)], (q1,q2))

In [16]:
len(seqs)

256

In [17]:
seqs[6]

[Id(q1)⊗ Id(q2),
 Y90m(q1)⊗ X(q2),
 X90(q1)⊗ Y90(q2),
 MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378>)⊗ MEAS(M-q2, shape_fun=<function autodyne at 0x112a47378>)]

Here we've created all the 16x16 experiments necessary for two-qubit process tomography.

### Sequence modification

All the experiments above are just python lists and can be altered in just the same way.  Say you wanted to change the first pulse in every experiment the most näive way would be to loop through all the lists and prepend it:

In [18]:
seqs = state_tomo([Y90m(q1)], (q1,))

In [19]:
seqs

[[Y90m(q1), Id(q1), MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378>)],
 [Y90m(q1), X90(q1), MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378>)],
 [Y90m(q1), Y90(q1), MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378>)],
 [Y90m(q1), X(q1), MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378>)]]

In [20]:
mod_seqs = [[Z90(q2)] + i for i in seqs]

In [21]:
mod_seqs

[[Z90(q2, frameChange=-1.5707963267948966),
  Y90m(q1),
  Id(q1),
  MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378>)],
 [Z90(q2, frameChange=-1.5707963267948966),
  Y90m(q1),
  X90(q1),
  MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378>)],
 [Z90(q2, frameChange=-1.5707963267948966),
  Y90m(q1),
  Y90(q1),
  MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378>)],
 [Z90(q2, frameChange=-1.5707963267948966),
  Y90m(q1),
  X(q1),
  MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378>)]]

This is a bit trivial since you could easily just add this to this list of pulses in state_tomography creation.  Fancier things can be done with the `itertools` and `operator` libraries.  I would suggest looking at some the tutorials and examples online using these. 

In [22]:
import itertools
import operator

Say you have the mod_seqs from above and want to drop all the `X(q1)` pulses from the sequences and wanted to append `x90m(q2)` to the front.  Not sure why you would want to do this, but let's just set that aside for now.  I could do this by:

In [23]:
mod_mod_seq = []
for seq in mod_seqs:
    results = itertools.filterfalse(lambda x: x==X(q1), seq)
    results = itertools.product([X90m(q2)], results)
    mod_mod_seq.append([*results])

In [24]:
mod_mod_seq

[[(X90m(q2), Z90(q2, frameChange=-1.5707963267948966)),
  (X90m(q2), Y90m(q1)),
  (X90m(q2), Id(q1)),
  (X90m(q2), MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378>))],
 [(X90m(q2), Z90(q2, frameChange=-1.5707963267948966)),
  (X90m(q2), Y90m(q1)),
  (X90m(q2), X90(q1)),
  (X90m(q2), MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378>))],
 [(X90m(q2), Z90(q2, frameChange=-1.5707963267948966)),
  (X90m(q2), Y90m(q1)),
  (X90m(q2), Y90(q1)),
  (X90m(q2), MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378>))],
 [(X90m(q2), Z90(q2, frameChange=-1.5707963267948966)),
  (X90m(q2), Y90m(q1)),
  (X90m(q2), MEAS(M-q1, shape_fun=<function autodyne at 0x112a47378>))]]

## Auspex

Tomograpy can be run just like any other sequence of experiments (RB, Ramsey, etc...) with Auspex.   

In [25]:
from auspex.exp_factory import QubitExpFactory as QEF

auspex-WARNING: 2018-06-25 16:29:01,611 ----> Could not load channelizer library; falling back to python methods.


In [26]:
seqs = state_tomo([Y90m(q1)], (q1,))
filename = compile_to_hardware(seqs, 'StateTomo/StateTomo')

Compiled 4 sequences.


In [27]:
exp = QEF.create(filename, repeats=2048) # take 2048 shots of each projection
# exp.run_sweeps()

auspex-WARNING: 2018-06-25 16:29:01,918 ----> Could not load aps2 library
auspex-WARNING: 2018-06-25 16:29:01,920 ----> Could not load aps2 library
auspex-WARNING: 2018-06-25 16:29:01,923 ----> Could not find the Holzworth driver.
auspex-WARNING: 2018-06-25 16:29:01,925 ----> Could not find the Holzworth driver.


The above should take 2048 shots of each experiment and depending on how your filter pipeline is setup, you might only have four averaged values or 2048x4 single shot values to process.

## Reconstruction

At the moment our tomography analysis is done in `QuantumTomography.jl`, a Julia package for this task.  This means we haven't implemented any analysis code in QGL or Auspex.  More information about QuantumTomography.jl can be found on the associated Github page.